In [4]:
# Libraries needed for NLP

!pip install gTTS
!pip install tflearn
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow.compat.v1 as tf
import numpy as np
import tflearn
import random
from gtts import gTTS
from IPython.display import Audio
tts = gTTS('hello joyjit')
tts.save('1.wav')
sound_file = '1.wav'
Audio(sound_file, autoplay=True)

import json

  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4097 sha256=26647819a93ccda0c0bcaae9a764f9b1774407c06b63f23746b6c8f14ae0ef55
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day","Namaste ji"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\n        },\n        {"tag": "fever",\n         "patterns": ["high body temperature", "b

In [0]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Namaste ji'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['high body temperature',
    'body temperature increasing',
    'high temperature',
    'very high temperature'],
   'responses': ['these are symptoms of fever',
    '

In [0]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

33 documents
9 classes ['deliveryoption', 'eyes', 'fever', 'goodbye', 'greeting', 'hours', 'menu', 'stomach', 'thanks']
58 unique stemmed words ["'s", 'ach', 'and', 'anyon', 'ar', 'body', 'bye', 'can', 'cours', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'ey', 'food', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'high', 'hom', 'hour', 'how', 'in', 'increas', 'is', 'ji', 'lat', 'main', 'me', 'menu', 'most', 'namast', 'op', 'opt', 'pain', 'provid', 'red', 'see', 'spec', 'stomach', 'tear', 'tel', 'temp', 'thank', 'that', 'the', 'ther', 'today', 'very', 'what', 'when', 'yo', 'you']


In [0]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [12]:
# resetting underlying graph data
tf.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 1.26305 | time: 0.016s
| Adam | epoch: 1000 | loss: 1.26305 - acc: 0.8872 -- iter: 32/33
Training Step: 5000  | total loss: 1.14457 | time: 0.021s
| Adam | epoch: 1000 | loss: 1.14457 - acc: 0.9086 -- iter: 33/33
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [0]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [15]:
# load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [0]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    tts = gTTS(random.choice(i['responses']))
                    tts.save('1.wav')
                    sound_file = '1.wav'
                    return Audio(sound_file, autoplay=True)

            results.pop(0)

In [22]:
classify('What are you hours of operation?')

[('hours', 0.85221)]

In [24]:
response('i have pain in eyes')

In [29]:
response('i am having high body temperature')

In [26]:
response('hi')

In [27]:
response('That is helpful')

In [28]:
response('Bye')

In [0]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [0]:
response('Can you please let me know the delivery options?')

We have home delivery options through UBER Eats and Zomato


In [0]:
response('What is menu for today?')

You can check various delicacies given in the food menu at www.mymenu.com


In [0]:
context

{'123': 'food'}

In [0]:
response("Hi there!", show_details=True)

context: 
tag: greeting
Good to see you again


In [0]:
response('What is menu for today?')

Today's special is Chicken Tikka
